In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import glob

## Read in antenna data from LOFAR svn

In [4]:
data = pd.DataFrame();
for stationfile in glob.glob('/Users/dijkema/opt/lofar/src/MAC/Deployment/data/Coordinates/ETRF_FILES/*/*.csv'):
    stationframe = pd.read_csv(stationfile, skip_blank_lines=True)
    stationframe["STATIONNAME"] = stationfile.split("/")[-2]
    # RS106 has rubbish lines with all zeros
    stationframe = stationframe[stationframe["NAME"].str[0]!="0"]
    data = pd.concat([data, stationframe], ignore_index=True)
    if (stationframe["NAME"].str[1]==".").any():
        print(stationfile)
        break

Separate the 'NAME' column into a type and a number:

Add a column `TYPE` with `LBA` and `HBA`:

In [5]:
data["TYPE"] = data["NAME"].str[0]+"BA"

For the centers (`CLBA`, `CHBA`, `CHBA0`, `CHBA1`), `TYPE` is the same as the name:

In [6]:
data.loc[data["NAME"].str[0]=="C", "TYPE"] = data.loc[data["NAME"].str[0]=="C", "NAME"].str.strip()

In [7]:
pd.value_counts(data[data["TYPE"].str[0]=="C"]["TYPE"])

CHBA     51
CLBA     51
CHBA1    24
CHBA0    24
Name: TYPE, dtype: int64

Add a column `ANTENNANUMBER` with the numeric part of `NAME`:

In [8]:
data["ANTENNANUMBER"] = 0

In [9]:
data.loc[data["NAME"].str[0]!="C", "ANTENNANUMBER"] = pd.to_numeric(data.loc[data["NAME"].str[0]!="C", "NAME"].str[1:])

Use `pyproj` to convert from ETRS x, y, z to ETRS lat, lon, height:

In [10]:
import pyproj

In [11]:
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

In [12]:
(data["ETRS-LON"], 
 data["ETRS-LAT"], 
 data["ETRS-HEIGHT"]) = pyproj.transform(ecef, lla,
                                         data["ETRS-X"].values,
                                         data["ETRS-Y"].values,
                                         data["ETRS-Z"].values)

## Make tiles

In [13]:
def getcoord(stationname, num):
    """
    Get the LON and LAT coordinates of an HBA-antenna
    """
    queryres = data[(data["STATIONNAME"]==stationname) & (data["ANTENNANUMBER"]==num) & (data["TYPE"]=="HBA")]
    if len(queryres) != 1:
        raise Exception("Problem finding antenna "+str(num)+" for station"+stationname+": found "+len(queryres))
    return np.array([queryres["ETRS-LON"].iloc[0], queryres["ETRS-LAT"].iloc[0]])

Get the offset between the tiles in a station (by comparing the position of tile 0 with that next to and below it).

In [14]:
def numbelowfirst(stationtype):
    """ Get the antenna number for when the number in a line is below zero.
    Stationtype must be CS, RS or other"""
    if stationtype=='CSHBA0':
        return 3
    if stationtype=='CSHBA1':
        return 27
    if stationtype=='RS':
        return 4
    else:
        return 6

In [15]:
dirxs = {}
dirys = {}
for stationname in pd.unique(data["STATIONNAME"]):
    subtypes = [""]
    if stationname[0:2] == "CS":
        subtypes = ["HBA0", "HBA1"]
    for subtype in subtypes:
        firstantenna = 0
        if subtype == "HBA1":
            firstantenna = 24;
        ant0coord = getcoord(stationname, firstantenna)
        dirxs[stationname+subtype] = getcoord(stationname, firstantenna+1) - ant0coord
        highy = numbelowfirst(stationname[0:2]+subtype)
        dirys[stationname+subtype] = getcoord(stationname, highy) - ant0coord

In [16]:
data["ETRS-WKT"] = ""
for index, rec in data.iterrows():
    if rec['TYPE'] != 'HBA':
        wkt = 'Point('+str(rec['ETRS-LON']) +' '+str(rec['ETRS-LAT'])+')';
    else:    
        xy=np.array([rec['ETRS-LON'],rec['ETRS-LAT']])

        # Find the offset vector between tiles
        subtype = ""
        if rec['STATIONNAME'][0:2] == "CS":
            if rec['ANTENNANUMBER'] >= 24:
                subtype = "HBA1"
            else:
                subtype = "HBA0"

        dirx = dirxs[rec['STATIONNAME'] + subtype]
        diry = dirys[rec['STATIONNAME'] + subtype]

        ul = xy - 0.5*dirx + 0.5*diry
        ur = xy + 0.5*dirx + 0.5*diry 
        lr = xy + 0.5*dirx - 0.5*diry 
        ll = xy - 0.5*dirx - 0.5*diry 

        wkt=('Polygon (('+str(ul[0])+' '+str(ul[1])+', '+ 
                          str(ur[0])+' '+str(ur[1])+', '+ \
                          str(lr[0])+' '+str(lr[1])+', '+ \
                          str(ll[0])+' '+str(ll[1])+', '+ \
                          str(ul[0])+' '+str(ul[1])+'))')

    data.set_value(index, "ETRS-WKT", wkt)

## Export to PostgreSQL

PostgreSQL wants column names to be lowercase, so make them lowercase:

In [17]:
data.columns = map(str.lower, data.columns)

In [18]:
from sqlalchemy import create_engine

In [19]:
engine = create_engine('postgresql://dijkema@localhost:5432/lofargeo')

In [20]:
engine.execute("delete from antennas;")
data.to_sql('antennas', engine, index=False, if_exists="append")

Add geometry

In [21]:
engine.execute('update antennas set geom = ST_SetSRID(ST_MakePoint("etrs-lon", "etrs-lat"), 4258)');

## Export to CSV

In [52]:
data.to_csv("allantennas.csv")

## Pretty station names

In [22]:
stations = pd.read_excel("/Users/dijkema/opt/lofarmap/stationsnamen.xlsx")

In [24]:
engine.execute("delete from stationnames");
stations.to_sql('stationnames', engine, index=False, if_exists='append')